# Chapter 4
# Finite-Dimensional Optimization

In [2]:
import numpy as np
from numpy import append, array, diagonal, tril, triu
from numpy.linalg import inv
from scipy.linalg import lu
#from scipy.linalg import solve
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot
np.random.seed(123)
import warnings
%matplotlib notebook
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D     #For 3D plotting
from matplotlib import cm   #colormap
from sympy import *
import sympy as sym
init_printing()

## 4.2 Newton-Raphson Method

A popular function used to test optimization routines is the so-called banana function: 

$$f = -100*(x_2-x_1^2)^2-(1-x_1)^2$$

so-called because its contours resemble bananas.

In [ ]:
f = lambda x,y:(-100*(y-x**2)**2-(1-x)**2)

n = 256
x = np.linspace(-0.25, 1.25, n)
y = np.linspace(-0.25, 1.25, n)
X,Y = np.meshgrid(x, y) 

plt.figure()
x0,y0 = 1,1
# use plt.contourf to filling contours
# X, Y and value for (X,Y) point
plt.contourf(X, Y, f(X, Y), 38, alpha=.75,cmap='bone')# cmap=plt.cm.hot)

# use plt.contour to add contour lines
C = plt.contour(X, Y, f(X, Y), 38, colors='black', linewidth=.5)

plt.clabel(C, inline=True, fontsize=10)
# plt.xticks(())
# plt.yticks(())

# set dot styles
plt.scatter([x0, ], [y0, ], s=50, color='b')



plt.xlim(-0.25, 1.25)
plt.ylim(-0.25, 1.25)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.contour3D(X, Y, f(X, Y), 50, cmap='binary', alpha=0.3)
ax.plot_surface(X, Y, f(X, Y), cmap=cm.coolwarm,alpha=0.7)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.set_zlim(-100,0)

plt.show()

The Newton-Raphson method for maximizing an objective function uses successive
quadratic approximations to the objective in the hope that the maxima of the approximants
will converge to the maximum of the objective. The Newton-Raphson
method is intimately related to the Newton method for solving rootfinding problems.

Indeed, the Newton-Raphson method is identical to applying Newton's method to
compute the root of the gradient of the objective function.

The Taylor series of $f(x)$ about the point  $x=x_0 + \epsilon$ is given by

$$f(x_0 + \epsilon) = f(x_0)+ f'(x_0) \epsilon +\frac{1}{2} f''(x_0) \epsilon^2 $$

$$f(x) = f(x^{(k)})+ f'(x^{(k)}) (x-x^{(k)}) + \frac{1}{2}(x-x^{(k)})^T  f''(x^{(k)}) (x-x^{(k)}) $$

Solving the first order condition

$$f'(x^{(k)})+ f''(x^{(k)}) (x-x^{(k)}) = 0$$

yields the iteration rule



$$x^{(k+1)} \leftarrow x^{(k)} -  [f''(x^{(k)})]^{-1} f'(x^{(k)}) $$

In theory, the Newton-Raphson method converges if $f$ is twice continuously difierentiable
and if the initial value of x supplied by the analyst is sufficiently close to a
local maximum of $f$ at which the **Hessian $f''$** is negative definite. There is, however,
no generally practical formula for determining what sufficiently close is.

The Newton-Raphson method can be robust to the starting
value if $f$ is well behaved, for example, if f is **globally concave**. The Newton-Raphson
method, however, can be very sensitive to starting value if the function is not globally
concave. Also, in practice, the **Hessian $f''$**  must be well-conditioned at the optimum,
otherwise rounding errors in the vicinity of the optimum can make it difficult to
compute a precise approximate solution.

The Newton-Raphson algorithm has numerous drawbacks. 

First, the algorithm
requires computation of both the first and second derivatives of the objective function.


Second, the Newton-Raphson algorithm offers no **guarantee** that the objective function
value may be increased in the direction of the Newton step. Such a guarantee is
available only if the Hessian **Hessian $f''(x^k)$** is **negative definite**; otherwise, one may actually
move towards a saddle point of f (if the Hessian is indefinite) or even a minimum (if
Hessian is **positive definite**).

For this reason, the Newton-Raphson method is rarely
used in practice, and then only if the objective function is **globally concave**.

## 4.3 Quasi-Newton Methods


Quasi-Newton methods employ a similar strategy to the Newton-Raphson method,
but **replace the Hessian of the objective function (or its inverse) with a negative
definite approximation, guaranteeing that function value can be increased in the direction
of the Newton step**. 

The most efficient quasi-Newton algorithms employ an
approximation to the inverse Hessian, rather than the Hessian itself, in order to avoid
performing a linear solve, and employ updating rules that do **not require second
derivative information** to ease the burden of implementation and the cost of computation.

In analogy with the Newton-Raphson method, quasi-Newton methods use a search
direction of the form

$$d^{(k)} = -B^{(k)} f'(x^{(k)})$$

where $B^{(k)}$ is an approximation to the **inverse Hessian** of f at the kth iterate $x^{(k)}$.
The vector $d^{(k)}$ is called the **Newton or quasi-Newton step**.

The more robust quasi-Newton methods do not necessarily take the full Newton
step, but rather shorten it or lengthen it in order to obtain improvement in the
objective function. This is accomplished by performing a line-search in which one
seeks a **step length $s > 0$** that maximizes or nearly maximizes $f (x^{(k)} + sd^{(k)})$. Given
the computed step length $s^{(k)}$, one updates the iterate as follows:

$$x^{(k+1)}=  x^{(k)} + s^{(k)}  d^{(k)}$$

Quasi-Newton methods differ in how the inverse Hessian approximation Bk is constructed
and updated. The simplest quasi-Newton method sets

$$B^{(k)} = - I $$

where I is  the identity matrix. This leads to a Newton step that is identical to the gradient of
the objective function at the current iterate:


$$d^{(k)} = f'(x^{(k)})$$

The choice of gradient as a step direction is intuitively appealing because the gradient
always points in the direction which, to a first order, promises the greatest increase in
$f$. For this reason, this quasi-Newton method is called the method of *steepest ascent.*


The steepest ascent method is simple to implement, but is numerically *less efficient*
in practice than competing quasi-Newton methods that *incorporate* information regarding
the **curvature of the objective function**.


The **most widely-used** quasi-Newton methods that employ **curvature information**
produce a sequence of inverse Hessian estimates that satisfy two conditions. 

**First,**
given that


$$d^{(k)} \approx f''^{-1}(x^{(k)})( f'(x^{(k)}+ d^{(k)} ) - f'(x^{(k)})  )$$

the **inverse Hessian estimate** $B^{(k)}$ is required to satisfy the so-called **quasi-Newton condition:**

$$d^{(k)} = B^{(k)}( f'(x^{(k)}+ d^{(k)} ) - f'(x^{(k)})  )$$

**Second,** the inverse Hessian estimate $B^{(k)}$  is required to be both **symmetric and
negative-definite**, as must be true of the inverse Hessian at a local maximum. The
negative definiteness of the Hessian estimate assures that the objective function value
can be inreased in the **direction of the Newton step**.


Two methods that satisfy the quasi-Newton and negative definiteness conditions
are the **Davidson-Fletcher-Powell (DFP)** and **Broyden-Fletcher-Goldfarb-Shano (BFGS)**
updating methods. The **DFP** method uses the updating scheme


$$B \leftarrow B + \frac{d d^T}{d^T u}  - \frac{B u u^T B}{u^T B u} $$


where

$$d = x^{(k+1)} - x^{(k)}$$

and 

$$u = f'(x^{(k+1)}) - f'(x^{(k)})$$

The **BFGS** method uses the update scheme


$$B \leftarrow B + \frac{1}{d^T u}( w d^T + d w^T  - \frac{w^T u}{d^T u}) d d^T $$

where 

$$w = d - B u$$



The BFGS algorithm is generally considered superior to DFP, although there
are problems for which DFP outperforms BFGS. However, except for the updating
formulae, the two methods are identical, so it is easy to implement both and give
users the choice.

In [ ]:
# list
step_methods = ['none','bhhh','bt','golden']                         
search_methods = ['Steepest','DFP','BFGS']                           

In [ ]:
# dictionary
# step_methods = {'none': _step_none,
#                      'bhhh': _step_bhhh,
#                      'bt': _step_bt,
#                      'golden': _step_golden
#                      }
# search_methods = {'steepest': _search_steepest,
#                        'dfp': _search_dfp
#                        'bfgs': _search_bfgs,
#                        }

In [ ]:
# def _search_bfgs(f, ff=None, u=None, d=None):
#         ud = np.inner(u, d)
#         w = d - B.dot(u)
#         wd = np.outer(w, d)
#         return  B+ ((wd + wd.T) - (np.inner(u, w) * np.outer(d, d)) / ud) / ud
#         # self.reset = False

# def _search_dfp(self, ff=None, u=None, d=None):
#         ud = np.inner(u, d)
#         v = B.dot(u)
#         return B+ np.outer(d, d) / ud - np.outer(v, v) / np.inner(u, v)
#         #self.reset = False

# def _search_steepest(self, ff, u=None, d=None):
#     return   -np.identity(k) / np.maximum(abs(fx0), 1)

### Find the best step size

In [ ]:
# The function optstep is not covered in the textbook.
# It is used to implement the qnewton routine.
errcode = False
def optstep(stepmeth,func, x0, fx0, g0, d, maxstep = 1000):
    """find the best step size"""
    # take multiple output of function
    A = func(x)
    k = x0.shape[0] # number of variables
    _is_there_jacobian = (type(A) is tuple) and (len(A) == 2)

    if _is_there_jacobian:
        #print('Jacobian was provided by user!')
        f = lambda z:  func(z)[0]
        
    # several step search method
    def _step_none(f, x0, fx0, d,maxstep):
        fx = f(x0 + d)
        if fx < fx0:
            s = 1
            errcode = False
            return s, f
        else:
            return _step_golden(f, x0, fx0, d,maxstep)

    def _step_bhhh(f, x0, fx0, g0, d,maxstep):
        # Intializations
        delta = 0.0001
        dg = -np.inner(g0, d)  # directional derivative
        tol1 = dg * delta
        tol0 = dg * (1 - delta)
        s, ds = 1, 1
        errcode = False

        # Bracket the cone
        for it in range(maxstep):
            x = x0 + s * d
            fs = f(x)
            temp = (fx0 - fs) / s
            if temp < tol0:
                ds *= 2
                s += ds
            else:
                break

        if (tol0 <= temp) and (temp <=tol1):
            return s, fs

        ds /= 2
        s -= ds
        it0 = it + 1

        # Then use bisection to get inside it
        for it in range(it0, maxstep):
            ds /= 2
            x = x0 + s * d
            fs =  f(x)
            temp = (fx0 - fs) / s
            if temp > tol1:
                s -= ds
            elif temp < tol0:
                s += ds
            else:
                return s, fs

        # If it has not returned yet, call _step_golden!
        return _step_golden(f, x0, fx0, d, maxstep)

    def _step_bt(f, x0, fx0, g0, d, maxstep):
        delta = 1e-4 # Defines cone of convergence; must be on (0,1/2)
        ub = 0.5     # Upper bound on acceptable reduction in s.
        lb = 0.1     # Lower bound on acceptable reduction in s.
        errcode = 0
        dg = -np.inner(d, g0)  # directional derivative
        tol1 = delta * dg
        tol0 = (1 - delta) * dg

        # full step
        s = 1
        fs = f(x0+d)
        if (fx0 - fs)   <= tol1:
            return s, fs

        # quadratic approximation
        s2, fs2 = s, fs
        s = -0.5 * dg / (-fs + fx0 - dg)
        s = max(s, lb)
        fs = f(x0 + s * d)
        temp = (-fs + fx0) / s
        if (tol0 <= temp) and (temp <= tol1):
            return s, fs

        # cubic approximation
        for it in range(3, maxstep):
            temp = (s - s2) * np.array([s * s, s2 * s2])
            temp = np.array([- fs + fx0 - dg * s, -fs2 + fx0 - dg * s2]) / temp
            a = temp[0] - temp[1]
            b = s * temp[1] - s2 * temp[0]
            s2 = s
            fs2 = fs
            if np.all(a == 0):  # quadratic fits exactly
                s = -0.5 * dg / b
            else:
                disc = b * b - 3 * a * dg
                if np.all(disc < 0):
                    errcode = 2
                    return s, fs  # complex root
                s = (np.sqrt(disc) - b) / (3 * a)

            s = np.maximum(np.minimum(s, ub * s2), lb * s2)  # ensures acceptable step size; cp(f, lb, up)
            fs = f(x0 + s * d)
            temp = (-fs + fx0) / s
            if np.all(tol0 <= temp) and np.all(temp <= tol1):
                return s, fs

        # If it has not returned yet, call _step_golden instead
        return _step_golden(f, x0, fx0, d,maxstep)

    def _step_golden(f, x0, fx0, d,maxstep):
        alpha1 = (3 - np.sqrt(5)) / 2
        alpha2 = (np.sqrt(5) - 1) / 2
        tol = 1.e-4
        tol *= alpha1*alpha2
        s = 1
        errcode = True
        niter = 0
        s0 = 0
        it = 0

        # Find a bracketing interval
        fs = f(x0 + d)
        if fx0 >= fs:
            lenght = alpha1
        else:
            for it in range(maxstep):
                s *= 2
                fl = fs
                fs = f(x0 + s*d)
                if fs <=fl:
                    lenght = alpha1 * (s - s0)
                    break
                else:
                    s0 /= 2

            if (it + 1) >= maxstep:
                s /= 2
                fs = fl
                return s, fs

        xl = x0 + (s + lenght) * d
        xs = x0 + (s - lenght) * d

        s -= lenght
        lenght *= alpha2  # lenght now measures relative distance between xl and xs

        fs = f(xs)
        fl = f(xl)

        # Golden search to find minimum
        while it < maxstep:
            it += 1
            if fs < fl:
                s -= lenght
                lenght *= alpha2
                xs = xl
                xl -= lenght * d
                fs = fl
                fl = f(xl)
            else:
                lenght *= alpha2
                s += lenght
                xl = xs
                xs += lenght * d
                fl = fs
                fs = f(xs)

            if lenght < tol:
                errcode = False
                break
        if fl > fs:
            fs = fl
            s -= lenght
        return s, fs
    
    # return resulted s and fx
    if stepmeth == None:
        return _step_none(f, x0, fx0, d,maxstep)
    elif stepmeth == "bhhh":
        return _step_bhhh(f, x0, fx0, g0, d,maxstep)
    elif stepmeth == "bt":
        return _step_bt(f, x0, fx0, g0, d,maxstep)
    elif stepmeth == "golden":
        return _step_golden(f, x0, fx0, d,maxstep)
        
    
    

In [ ]:
# Banana function
def f(x):
    y = (-100*(x[1]-x[0]**2)**2-(1-x[0])**2)
    dy = np.array([2*(1-x[0])+400*(x[1]-x[0]**2)*x[0],  -200*(x[1]-x[0]**2)])
    return y,dy

The script assumes that the user
has written a Python routine $f$ that evaluates the function at an arbitrary point and
that the user has specified a starting point $x$, an initial guess for the inverse Hessian
$A$, a convergence tolerance tol, and a limit on the number of iterations maxit. The
script uses an auxiliary algorithm optstep to determine the step length (discussed
in the next section). The algorithm also offers the user a choice on how to select the
search direction, searchmeth (1-steepest ascent, 2-DFP, 3-BFGS).

https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

In [ ]:
#         if self.x0 is None or self.x0[0] is None:
#             raise ValueError('Initial value is required to solve a OP, none provided!')

In [ ]:
#In this cell we select the search method and the step method

x_list = list()#  sequence of solutions of x for plotting

x0 = np.array([1.,0.]) # initial value for x

maxit, maxstep, tol,eps0, eps1,all_x  = 10000, 10000, 1/10000,1.0,1.e-12 ,False # keyword arguments

x_list = [x0] # first x
searchmeth =2 #  pick a search method.
stepmeth = "bt"

In [ ]:
x = x0 # initialize
k = x.shape[0] # number of variables
eps = np.spacing(1) # epsilon

A = f(x) # tuble of multiple outputs from object function

_is_there_jacobian = (type(A) is tuple) and (len(A) == 2)

# get first fx and g. object value and gradient/hessian value. 
if _is_there_jacobian:
    print('Jacobian was provided by user!')
    fx0,g0 = f(x)
else:    
    print('Jacobian was not provided by user!')
    fx0 = f(x)
    try:
        g0 = jacobian(f,x) # customized jacobian function
    except NameError:
        print("Jacobian function Not in scope!\n Using identity matrix as Jacobian matrix")
        g0 = np.identity(k)
    else:
        print("Jacobian function In scope!")

B = None # inversed Hessian matrix        
        
if B is None:
    B =  -np.identity(k) / np.maximum(abs(fx0), 1)   # using identity matrix as Hessian
    print("Hessian is not provided and reset as normalized identity matrix! so steepest ascent") # steepest ascent
    
d = -np.dot(B, g0)  # search direction    

In [ ]:
# This cell shows one update based on a specific method
s, fx = optstep("bt", f, x, fx0, g0, d, maxstep)

s,fx

In [ ]:
# This cell shows one update based on a different method
s, fx = optstep("golden", f, x0, fx0, g0, d, maxstep)
s,fx

In [ ]:
# This cell solves the maximization problem, relying on the functions defined above

if np.linalg.norm(g0) < eps: # similar to np.all(g0<eps)
    #break #return x
    print("g0 is less than eps")
if np.all(g0 < eps): # check conditions
    #break #return x
    print("g0 is less than eps")
print("Solving optimization by using {} search method and {} step method".format(search_methods[searchmeth-1].capitalize(), stepmeth)) 

print("Start iteration......")


for it in range(maxit):
    
    
    d = -np.dot(B, g0)  # search direction
    
    if (np.inner(d, g0) / (np.inner(d, d))) < eps1:  # must go uphill
        B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
        d = g0 / np.maximum(np.abs(fx0), 1)  # steepest ascent

    s, fx = optstep("bt", f, x, fx0, g0, d, maxstep)
    
    if fx <= fx0:
        
        warnings.warn('Iterations stuck in qnewton')
        # break #x # return x
        # reset Hessian and d.
        B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
        d = g0.T / np.maximum(abs(fx0), 1)  # steepest ascent
        s, fx = optstep("bt" , f, x, fx0, g0, d, maxstep)
        if errcode:
            warnings.warn('Cannot find suitable step in qnewton')
            # return x
            # reset to 1 and fx0
            s, fx =  1, fx0
    
    
    d *= s
    x = x + d
    
    x_list.append(x.copy())

    if np.any(np.isnan(x) | np.isinf(x)):
        raise ValueError('NaNs or Infs encountered')
    
    # update fx and g
    if _is_there_jacobian:
        #print('Jacobian was provided by user!')
        fx,g = f(x)
    else:    
        print('Jacobian was not provided by user!')
        fx = f(x)
        try:
            g = jacobian(f,x)
        except NameError:
            print("Jacobian function Not in scope!\n Using identity matrix as Jacobian matrix")
            g = np.identity(k)
        else:
            print("Jacobian function In scope!")
    

    # Test convergence using Marquardt's criteria and gradient test
    if ((fx - fx0) / (abs(fx) + eps0) < tol and
            np.all(np.abs(d) / (np.abs(x) + eps0) < tol)) or\
            np.all(np.abs(g) < eps):
            print("Meet the tol. x: ", x)
            break
#         #return x

#     if np.all( np.abs(d)/(np.abs(x) + eps0)< tol) or np.all(np.abs(g) < eps):
#         print("Meet the tol. x: ", x)
#         break
        

    # Update inverse Hessian
    u = g - g0  # change in Jacobian
    ud = np.inner(u, d)
    
    
    #print("Please specify one search method: 1:steepest ascen;2: DFP;3:BFGS")
    if np.all(np.abs(ud) < eps):
        B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
    else:
        if searchmeth == 1 and np.abs(ud) < eps: # steepest ascent
            B =  -np.identity(k) / np.maximum(abs(fx), 1)
        elif searchmeth == 2: # DFP
            v = B.dot(u)
            B += np.outer(d, d) / ud - np.outer(v, v) / np.inner(u, v) 
        elif searchmeth == 3: # BFGS
            w = d - B.dot(u)
            wd = np.outer(w, d)
            B += ((wd + wd.T) - (np.inner(u, w) * np.outer(d, d)) / ud) / ud
#         else:
#             print("Please specify one search method: 1:steepest ascen;2: DFP;3:BFGS")

    # Update iteration
    fx0 = fx
    g0 = g
    print("Completed {}th iteration...".format(it))   
    
#print("x list: " +  for str(x) in x_list)    
if it > maxit:
    warnings.warn('Maximum iterations exceeded in qnewton')

In [ ]:
# This cell reports the whole sequence of guesses/iterations
x_list


In [ ]:
x0 = np.array([1.,0.]) # initial value for x

B = None # inverse Hessian matrix        
        
if B is None:
    B =  -np.identity(k) / np.maximum(abs(fx0), 1)   # using identity matrix as Hessian
    print("Hessian is not provided and reset as normalized identity matrix! so steepest ascent") # steepest ascent


f,x0,B

In [ ]:
#This part of the code is a general implementation of quasi-Netwon methods

def myqnewton(f, x0, B=None, searchmeth = 3, stepmeth = "bt", maxit = 10000, maxstep = 10000, tol = 1/100000,\
              eps = np.spacing(1), eps0 =1.0, eps1 = 1.e-12, all_x = False):
    '''
    maxit, maxstep, tol,eps0, eps1  = 10000, 10000, 1/10000,1.0,1.e-12
    f: object function and jacobian
    x0: initial value
    all_x: if we collect x value for plotting
    '''
    x = x0
    if all_x:
        x_list = [x0]
        
    k = x.shape[0] # number of variables
    A = f(x)
    _is_there_jacobian = (type(A) is tuple) and (len(A) == 2)

    if _is_there_jacobian:
        print('Jacobian was provided by user!')
        fx0,g0 = f(x)
    else:    
        print('Jacobian was not provided by user!')
        fx0 = f(x)
        try:
            g0 = jacobian(f,x)
        except NameError:
            print("Jacobian function Not in scope!\n Using identity matrix as Jacobian matrix")
            g0 = np.identity(k)
        else:
            print("Jacobian function In scope!")    
        
    if np.all(np.abs(g0) < eps): # similar to np.all(g0<eps)
        print("abs(g0)< eps...")
        return x
    
    print("Solving optimization by using {} search method and {} step method".format(search_methods[searchmeth-1].capitalize(), stepmeth)) 

    print("Start iteration......")
    
    #B = None # inversed Hessian matrix        
        
    if B is None:
        B =  -np.identity(k) / np.maximum(abs(fx0), 1)   # using identity matrix as Hessian
        print("Hessian is not provided and reset as normalized identity matrix! so steepest ascent") # steepest ascent

    for it in range(maxit):

        d = -np.dot(B, g0)  # search direction, initial d
        
        # https://github.com/randall-romero/CompEcon-python/blob/master/compecon/optimize.py
        if (np.inner(d, g0) / (np.inner(d, d))) < eps1:  # must go uphill
            B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
            d = g0 / np.maximum(np.abs(fx0), 1)  # steepest ascent
        # optimize search step length
        s, fx = optstep(stepmeth ,f, x, fx0, g0, d, maxstep)

        if fx <= fx0:

            warnings.warn('Iterations stuck in qnewton')
            #return x
            # reset Hessian and d.
            B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
            d = g0.T / np.maximum(abs(fx0), 1)  # steepest ascent
            s, fx = optstep("bt" ,f, x, fx0, g0, d, maxstep)
            if errcode:
                warnings.warn('Cannot find suitable step in qnewton')
                # return x
                # reset to 1 and fx0
                s, fx =  1, fx0

        # update d and x
        d *= s
        x = x + d
        # keep record of x sequence in list
        if all_x:
            x_list.append(x.copy())

        if np.any(np.isnan(x) | np.isinf(x)):
            raise ValueError('NaNs or Infs encountered')

        #  update fx and g again
        if _is_there_jacobian:
            #print('Jacobian was provided by user!')
            fx,g = f(x)
        else:    
            print('Jacobian was not provided by user!')
            fx = f(x)
            try:
                g = jacobian(f,x)
            except NameError:
                print("Jacobian function Not in scope!\n Using identity matrix as Jacobian matrix")
                g = np.identity(k)
            else:
                print("Jacobian function In scope!")

        # Test convergence using Marquardt's criteria and gradient test
        if ((fx - fx0) / (abs(fx) + eps0) < tol and
                np.all(np.abs(d) / (np.abs(x) + eps0) < tol)) or\
                np.all(np.abs(g) < eps):
                print("Meet the tol. x: ", x)
                #break
                if all_x:
                    return x, x_list
                else:
                    return x

        # Update inverse Hessian
        u = g - g0  # change in Jacobian
        ud = np.inner(u, d)

        # pick a search method
        #print("Please specify one search method: 1:steepest ascen;2: DFP;3:BFGS")
        if np.all(np.abs(ud) < eps):
            B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
        else:
            if searchmeth == 1 and np.abs(ud) < eps: # steepest ascent
                B =  -np.identity(k) / np.maximum(abs(fx), 1)
            elif searchmeth == 2: # DFP
                v = B.dot(u)
                B += np.outer(d, d) / ud - np.outer(v, v) / np.inner(u, v) 
            elif searchmeth == 3: # BFGS
                w = d - B.dot(u)
                wd = np.outer(w, d)
                B += ((wd + wd.T) - (np.inner(u, w) * np.outer(d, d)) / ud) / ud

        # Update iteration
        fx0 = fx
        g0 = g
        print("Completed {}th iteration...".format(it))   

    # end of iteration if exceed the maxit
    if it >= maxit:
        warnings.warn('Maximum iterations exceeded in qnewton')
        return x

In [ ]:
# Here and in the next three cells we compare the perfomance of different methods using a benchmark case.
x0 = np.array([1.,0.]) # initial value for x
B = None # inverse Hessian matrix   
myqnewton(f, x0, B, searchmeth = 3,stepmeth = "bt" ,maxit = 10000, maxstep = 10000,tol = 1/100000,\
              eps = np.spacing(1),eps0 =1.0, eps1 = 1.e-12, all_x = False)

In [ ]:
x0 = np.array([1.,0.]) # initial value for x
B = None # inverse Hessian matrix    
myqnewton(f, x0, B, searchmeth = 2,stepmeth = "bt" ,maxit = 10000, maxstep = 10000,tol = 1/100000,\
              eps = np.spacing(1),eps0 =1.0, eps1 = 1.e-12, all_x = False)

In [ ]:
x0 = np.array([1.,0.]) # initial value for x
myqnewton(f, x0,B = None)

In [ ]:
x0 = np.array([1.,0.]) # initial value for x
B = None # inverse Hessian matrix   
myqnewton(f, x0, B, searchmeth = 2, stepmeth = "bt", maxit = 10000, maxstep = 10000, tol = 1/100000,\
              eps = np.spacing(1), eps0 =1.0, eps1 = 1.e-12, all_x = True)

In [ ]:
x0 = np.array([1.,0.]) # initial value for x
B = None # inverse Hessian matrix   
myqnewton(f, x0, B, searchmeth =1, stepmeth = "bt", maxit = 10000, maxstep = 10000, tol = 1/100000,\
              eps = np.spacing(1), eps0 =1.0, eps1 = 1.e-12, all_x = False)

## 4.4 Line Search Methods
Just as was the case with rootfinding problems, it is not always best to take a full
Newton step. In fact, it may be better to either stop short or move past the Newton
step. If we view the Newton step as defining a *search direction*, performing a onedimensional
search in that direction will generally produce improved results.

http://reference.wolfram.com/language/tutorial/UnconstrainedOptimizationLineSearchMethods.html

https://en.wikipedia.org/wiki/Line_search

A number of diffierent line
search methods are used in practice, including the golden search method. 

The **golden search** algorithm is very reliable, but computationally inefficient. Two alternative
schemes are typically used in practice to perform line searches. 

The first, known as
the **Armijo search**, is similar to the backstepping algorithm used in rootfinding and complementarity problems. The idea is to find the minimum power $j$ such that:


In [ ]:
# https://github.com/smwade/ACME-2/blob/master/line_search/solutions.py

def backtracking(f, slope, x, p, a=1, rho=.9, c=10e-4):
    """Perform a backtracking line search to satisfy the Armijo Conditions.
    Parameters:
        f (function): the twice-differentiable objective function.
        slope (float): The value of grad(f)^T p.
        x (ndarray of shape (n,)): The current iterate.
        p (ndarray of shape (n,)): The current search direction.
        a (float): The intial step length. (set to 1 in Newton and
            quasi-Newton methods)
        rho (float): A number in (0,1).
        c (float): A number in (0,1).
    
    Returns:
        (float) The computed step size satisfying the Armijo condition.
    """
    while f(x + a*p) > f(x) + c * a * slope:
        a = float(rho * a)
    return a



Another widely-used approach, known as **Goldstein search**, is to find any value of $s$ that satisfies

A simple strategy for locating an acceptable point is to first find a point in or
above the cone using step doubling (doubling the value of s at each iteration). If a
point above the cone is found first, we have a bracket within which points in the cone
must lie. We can then narrow the bracket using the golden search method. We call this the bhhhstep approach.

Another approach, stepbt, checks to see if $s = 1$ is in the cone and, if so, maximizes
a quadratic approximation to the objective function in the Newton direction
constructed from knowledge of $f(x)$, $f(x)d$ and $f(x + d)$. If the computed step $s$ is
acceptable, it is taken. Otherwise, the algorithm iterates until an acceptable step is
found using a cubic approximation to the objective function in the Newton direction
constructed from knowledge of $f(x)$, $f(x)d$, $f(x + s(j+1)d)$ and $f(x + s(j)d)$. stepbt
is fast and generally gives good results. It is recommended as the default lines search
procedure for general maximization algorithms.

##  4.5 Special Cases
Two special cases arise often enough in economic practice (especially in econometrics)
to warrant additional discussion. Nonlinear least squares and the maximum likelihood
problems have objective functions with special structures that give rise to their
own special quasi-Newton methods. The special methods differ from other Newton
and quasi-Newton methods only in the choice of the matrix used to approximate the
Hessian. Because these problems generally arise in the context of statistical applications,
we alter our notation to conform with the conventions for those applications.
The optimization takes place with respect to a $k$-dimensional parameter vector $\theta$ and
$n$ will refer to the number of observations.

## 4.6 Scipy Optimization

Often we need to find the maximum or minimum of a particular function $f(x)$ where $f$ is a scalar function but $x$ could be a vector. Typical applications are the minimisation of entities such as cost, risk and error, or the maximisation of productivity, efficiency and profit. Optimisation routines typically provide a method to minimise a given function: if we need to maximise $f(x)$ we create a new function $g(x)$ that reverses the sign of $f$, i.e. $g(x)= − f(x)$ and we minimise $g(x)$.

Below, we provide an example showing (i) the definition of the test function and (ii) the call of the `scipy.optimize.fmin` function which takes as argument a function $f$ to minimise and an initial value $x_0$ from which to start the search for the minimum, and which returns the value of $x$ for which $f(x)$ is (locally) minimised. Typically, the search for the minimum is a local search, i.e. the algorithm follows the local gradient. We repeat the search for the minimum for two values ($x_0 = 1.0$ and $x_0 = 2.0$, respectively) to demonstrate that depending on the starting value we may find different minimar of the function f.

The majority of the commands (after the two calls to `fmin`) creates the plot of the function, the start points for the searches and the minima obtained:

In [ ]:
from numpy import arange, cos, exp
from scipy.optimize import fmin

def f(x):
    return cos(x) - 3 * exp( -(x - 0.2) ** 2)

# find minima of f(x),
# starting from 1.0 and 2.0 respectively
minimum1 = fmin(f, 1.0)
print("Start search at x=1., minimum is", minimum1)
minimum2 = fmin(f, 2.0)
print("Start search at x=2., minimum is", minimum2)

# plot function
plt.figure()
x = arange(-10, 10, 0.1)
y = f(x)
plt.plot(x, y, label='$\cos(x)-3e^{-(x-0.2)^2}$')
plt.xlabel('x')
plt.grid()
plt.axis([-5, 5, -2.2, 0.5])

# add minimum1 to plot
plt.plot(minimum1, f(minimum1), 'vr',
           label='minimum 1')
# add start1 to plot
plt.plot(1.0, f(1.0), 'or', label='start 1')

# add minimum2 to plot
plt.plot(minimum2,f(minimum2),'vg',\
           label='minimum 2')
# add start2 to plot
plt.plot(2.0,f(2.0),'og',label='start 2')

plt.legend(loc='lower left')

## References


- Optimization and Solving Systems of Equations in Julia

https://github.com/pkofod/JC2017



https://www.youtube.com/watch?v=E_UlaGoObTw


Using optimization routines from scipy and statsmodels
http://people.duke.edu/~ccc14/sta-663-2017/14C_Optimization_In_Python.html
http://people.duke.edu/~ccc14/sta-663-2017/14B_Multivariate_Optimization.html#convexity


http://people.duke.edu/~ccc14/sta-663-2017/14B_Multivariate_Optimization.html